In [1]:
# Load dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
 
[WDM] - Driver [C:\Users\DELGCA01\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


## NASA Mars page load

In [3]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

## Scrape NASA Mars News Titles and Paragraphs

In [4]:
news_titles = soup.find('div', class_="content_title")
news_titles.text.strip()

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

In [5]:
news_paragraph = soup.find('div',class_='rollover_description_inner')
news_paragraph.text.strip()

'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'

## Scrape JPL Mars Space Images

In [6]:
base_url = "https://www.jpl.nasa.gov"
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
html = browser.html
image_page = bs(html,'html.parser')

In [7]:
featured_image = image_page.find('img',class_='thumb')
featured_image_url = base_url + featured_image['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24247-640x350.jpg'

## Mars Facts scrape site

In [8]:
#Navigate to the next site to scrape
mars_facts_url = "https://space-facts.com/mars/"
mars_facts = pd.read_html(mars_facts_url)[2]
mars_facts.columns =["Data","Value"]
mars_facts


,Data,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [9]:
mars_facts_table_html = mars_facts.to_html()
print(mars_facts_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Data</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>R

## Mars Hemisphere Images Scrape Site

In [10]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html,'html.parser')


In [11]:
# Get the HTML containing the titles and put into a list
title_list = soup.find_all('div', class_='description')

In [12]:
# Loop through the 'div' objects and scrape the titles and urls of images
# Create a list to store the dictionaries
hemisphere_image_urls = []
for title in title_list: 
    # Navigate browser to page then click on title link to image page
    browser.visit(url)
    browser.click_link_by_partial_text(title.a.h3.text)
    
    # Grab the destination page html and make into BeautifulSoup Object
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Parse the imgage source(src) relative url and then append to domain name
    # for absolute url
    img_url_list = soup.find('img', class_='wide-image')
    image_url = f"https://astrogeology.usgs.gov{img_url_list['src']}"
    
    # Create Dictionary with returned values and add dict to hemi_image_urls list
    post = {
            'title': title.a.h3.text,
            'image_url': image_url
            }
    hemisphere_image_urls.append(post)

In [13]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

## Close Chrome Browser

In [14]:
browser.quit()